### 필요한 라이브러리들

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup as bs
import pandas as pd

### 1) 제품군 선택해서 get에 해당 제품군에 대한 링크 할당
- 선택한 제품군을 Bestselling 순으로 정렬한 페이지 get
- TV : 'https://www.amazon.in/s?bbn=1389396031&rh=n%3A1389396031%2Cp_89%3ALG&dc&qid=1724652002&rnid=3837712031&ref=lp_1389396031_nr_p_89_0'
- 세탁기 : 'https://www.amazon.in/s?bbn=1380369031&rh=n%3A1380369031%2Cp_89%3ALG&dc&qid=1724689251&rnid=3837712031&ref=lp_1380369031_nr_p_89_2'
- 에어컨 : 'https://www.amazon.in/s?bbn=3474656031&rh=n%3A3474656031%2Cp_89%3ALG&dc&qid=1724689377&rnid=3837712031&ref=lp_3474656031_nr_p_89_2'
- 냉장고 : 'https://www.amazon.in/s?bbn=1380365031&rh=n%3A1380365031%2Cp_89%3ALG&dc&qid=1724689330&rnid=3837712031&ref=lp_1380365031_nr_p_89_2'

In [117]:
driver=webdriver.Chrome()
driver.get('https://www.amazon.in/s?bbn=1380365031&rh=n%3A1380365031%2Cp_89%3ALG&dc&qid=1724689330&rnid=3837712031&ref=lp_1380365031_nr_p_89_2')
driver.maximize_window()

### 2) 각각의 제품 모델에 대한 상세 페이지 접근
- url 수집 및 저장
- 최대 24개의 제품 모델

In [29]:
soup = bs(driver.page_source, 'lxml')
# url 접근 위한 select
links = soup.select('h2 > a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
# 모델명 데이터 수집 위한 select
names = soup.select('span.a-size-base-plus.a-color-base.a-text-normal')

link_list = []
for link in links :
    link_list.append("https://www.amazon.in"+link['href'])

name_list = []
for name in names :
    name_list.append(name.text)



In [30]:
name_list # 모델명 확인용 
link_list # url 확인용

['https://www.amazon.in/LG-Inverter-Frost-Free-Refrigerator-GL-I292RPZX/dp/B08X72GY5Q/ref=sr_1_1?dib=eyJ2IjoiMSJ9.jOwKFjivYKJi2bAxYI9uukBYVgpIIzHo0fnIX8Ddw9BQEz3K6WH4I0EHt5WMlnRHJkDZgQXMNQksx1bJW-aTd9__etxaUjOQLsp0lIyREx8I6vSSo4fuQPSby-snN06HQ7eR0fE3w8IE5VUoZdBL13yG7wQcLS0WSUZkSMH9KKPwXtYiyH-u7ObNyF43NaNsBdjH1tJMIX3FuJLsP_P2qSGaEXcHfBz0Sc3pj0IAdTVcP_mCowdhEMGCjDPlYFCtg6mfBTutecMCDlBsCc44lHEhDxNpShCuEshp5mhDKwE.q9qsEcOguwIlzPA30-x6vt15pZ34vnp43js0m3wDeq4&dib_tag=se&qid=1724830811&refinements=p_89%3ALG&rnid=3837712031&s=kitchen&sr=1-1',
 'https://www.amazon.in/LG-Inverter-Direct-Cool-Refrigerator-GL-D201APZU/dp/B0BRQ9L7P1/ref=sr_1_2?dib=eyJ2IjoiMSJ9.jOwKFjivYKJi2bAxYI9uukBYVgpIIzHo0fnIX8Ddw9BQEz3K6WH4I0EHt5WMlnRHJkDZgQXMNQksx1bJW-aTd9__etxaUjOQLsp0lIyREx8I6vSSo4fuQPSby-snN06HQ7eR0fE3w8IE5VUoZdBL13yG7wQcLS0WSUZkSMH9KKPwXtYiyH-u7ObNyF43NaNsBdjH1tJMIX3FuJLsP_P2qSGaEXcHfBz0Sc3pj0IAdTVcP_mCowdhEMGCjDPlYFCtg6mfBTutecMCDlBsCc44lHEhDxNpShCuEshp5mhDKwE.q9qsEcOguwIlzPA30-x6vt15pZ34vnp43js0m3wDeq4&

In [118]:
len(link_list)

24

+ 리뷰별 처리된 데이터 수집

In [31]:
import pandas as pd
import re

#전처리 함수
def appendList(spect_list):

    keyword_list.append(re.findall(r'"(.*?)"', spect_list[0]))

    m=int(spect_list[0][:spect_list[0].find(' ')])
    mention_list.append(m)
    
    p=int(spect_list[1][:spect_list[1].find(' ')])
    positive_list.append(p)
    
    n=int(spect_list[2][:spect_list[2].find(' ')])
    negative_list.append(n)

    nega_percent_list.append(round(n/m,2))
    
    comment_list.append(spect_list[3])
    
    example_list=[]
    
    for line in spect_list[4:]:
        example_list.append(re.findall(r'"(.*?)"', line)[0])
        
    examples_list.append(example_list)    

In [32]:
product_name_list=[]
keyword_list=[]
mention_list=[]
positive_list=[]
negative_list=[]
nega_percent_list=[]
comment_list=[]
examples_list=[]
text_list=[]

for link in link_list:
    driver=webdriver.Chrome()
    #드라이버에 리뷰페이지 링크 넣기
    driver.get(link)
    driver.maximize_window()


    # #제품명
    # id = driver.find_elements(By.CLASS_NAME,"a-size-base.prodDetAttrValue")
    # name = id[1].text
    
    for i in range(10): #가능한 만큼 for문 돌아가면서 클릭함
        try:
            #키워드 버튼 클릭
            aspect = driver.find_element(By.ID,f"aspect-button-0-{i}")
            aspect.click()
            
            #클릭 상태에서 본문 뽑기
            spect = driver.find_element(By.ID,f"aspect-bottom-sheet-0-{i}")
            text_list.append(spect.text)

            #전처리
            for txt in text_list: 
                if txt !='':
                    spect_list=txt.split('\n')
                    appendList(spect_list)
                    product_name_list.append(name) #길이 지정해주기 위해 제품명은 따로 append
                
        except:
            break

- 최종 결과 데이터프레임

In [33]:
data = {
    'product_name': product_name_list,
    'keyword': keyword_list,
    'mention': mention_list,
    'positive': positive_list,
    'negative': negative_list,
    'neg_percent': nega_percent_list,
    'comment': comment_list,
    'examples': examples_list
}

df = pd.DataFrame(data)
df

,product_name,keyword,mention,positive,negative,neg_percent,comment,examples
0,GL-I292RPZX,[Quality],294,233,61,0.21,Customers like the quality of the refrigerator...,"[Good product, Nice productPacking and quality..."
1,GL-I292RPZX,[Quality],294,233,61,0.21,Customers like the quality of the refrigerator...,"[Good product, Nice productPacking and quality..."
2,GL-I292RPZX,[Value],62,54,8,0.13,Customers like the value of the refrigerator. ...,[Best budgeted double door fridge with efficie...
3,GL-I292RPZX,[Quality],294,233,61,0.21,Customers like the quality of the refrigerator...,"[Good product, Nice productPacking and quality..."
4,GL-I292RPZX,[Value],62,54,8,0.13,Customers like the value of the refrigerator. ...,[Best budgeted double door fridge with efficie...
...,...,...,...,...,...,...,...,...
4273,GC-B22FTLPL,[Quality],11,8,3,0.27,Customers like the quality of the refrigerator...,"[...Fridge is above and freezer is below, that..."
4274,GC-B22FTLPL,[Size],9,9,0,0.00,Customers are satisfied with the size of the r...,[...Food remains fresh for days together. Vege...
4275,GC-B22FTLPL,[Value],4,4,0,0.00,Customers appreciate the value of the refriger...,"[...Overall very fine and excellent product, w..."
4276,GC-B22FTLPL,[Cooling],3,3,0,0.00,Customers are satisfied with the cooling of th...,[...Freezer space is too much for vegetarians....


In [114]:
df = pd.read_csv('./Amazon_LG_Refrigerator_insight_review.csv',index_col=0)
df

df = df.drop_duplicates(subset=[df.columns[0], df.columns[2],df.columns[3],df.columns[4]]).reset_index(drop=True)
df

,product_name,keyword,mention,positive,negative,neg_percent,comment,examples
0,GL-I292RPZX,['Quality'],294,233,61,0.21,Customers like the quality of the refrigerator...,"['Good product', 'Nice productPacking and qual..."
1,GL-I292RPZX,['Value'],62,54,8,0.13,Customers like the value of the refrigerator. ...,['Best budgeted double door fridge with effici...
2,GL-I292RPZX,['Appearance'],54,49,5,0.09,Customers are satisfied with the appearance of...,['Appearance was very good from front side but...
3,GL-I292RPZX,['Cooling'],118,62,56,0.47,Customers are mixed about the cooling of the r...,"['More space inside, door cooling is excellent..."
4,GL-I292RPZX,['Size'],68,44,24,0.35,Customers are mixed about the size of the refr...,"['More space inside, door cooling is excellent..."
...,...,...,...,...,...,...,...,...
633,GC-B22FTLPL,['Quality'],7,7,0,0.00,Customers are satisfied with the quality of th...,['This refrigerator is really good and has mos...
634,GC-B22FTLPL,['Performance'],3,3,0,0.00,Customers are satisfied with the performance o...,['It is convertible. With super quite working....
635,GC-B22FTLPL,['Quality'],11,8,3,0.27,Customers like the quality of the refrigerator...,"['...Fridge is above and freezer is below, tha..."
636,GC-B22FTLPL,['Value'],4,4,0,0.00,Customers appreciate the value of the refriger...,"['...Overall very fine and excellent product, ..."


In [52]:
# df.to_csv('Amazon_LG_TV_insight_review.csv')
# df.to_csv('Amazon_LG_Washing_Machines_insight_review.csv')
# df.to_csv('Amazon_LG_Air_Conditioner_insight_review.csv')
df.to_csv('Amazon_LG_Refrigerator_insight_review.csv')

target_df = pd.DataFrame({
    'Product Name': name_list,
    'Link': link_list
})

# target_df.to_csv('Amazon_LG_TV_insight_target.csv')
# target_df.to_csv('Amazon_LG_Washing_Machines_insight_target.csv')
# target_df.to_csv('Amazon_LG_Air_Conditioner_insight_target.csv')
target_df.to_csv('Amazon_LG_Refrigerator_insight_target.csv')

- 엑셀 처리하기

In [84]:
columns = ['product_name', 'mention', 'positive', 'negative', 'neg_percent','comment','examples']

# Create an empty DataFrame with the specified columns
df_max = pd.DataFrame(columns=columns)

In [119]:
len(list(set(df['product_name'].unique())))

13

In [120]:
pd.set_option('display.max_rows', None)

In [121]:
df[df['product_name']=='GC-B22FTLPL']

,product_name,keyword,mention,positive,negative,neg_percent,comment,examples
554,GC-B22FTLPL,['Quality'],294,233,61,0.21,Customers like the quality of the refrigerator...,"['Good product', 'Nice productPacking and qual..."
555,GC-B22FTLPL,['Value'],62,54,8,0.13,Customers like the value of the refrigerator. ...,['Best budgeted double door fridge with effici...
556,GC-B22FTLPL,['Appearance'],54,49,5,0.09,Customers are satisfied with the appearance of...,['Appearance was very good from front side but...
557,GC-B22FTLPL,['Cooling'],118,62,56,0.47,Customers are mixed about the cooling of the r...,"['More space inside, door cooling is excellent..."
558,GC-B22FTLPL,['Size'],68,44,24,0.35,Customers are mixed about the size of the refr...,"['More space inside, door cooling is excellent..."
559,GC-B22FTLPL,['Noise'],61,25,36,0.59,Customers are mixed about the noise of the ref...,['...items inside fridge but vegetable tray is...
560,GC-B22FTLPL,['Service'],38,21,17,0.45,Customers are mixed about the service. Some me...,['Great service and good product . Thoroughly ...
561,GC-B22FTLPL,['Scratching'],44,6,38,0.86,Customers are dissatisfied with the scratching...,['...hour I just opened the boxes then I found...
562,GC-B22FTLPL,['Quality'],249,222,27,0.11,Customers are satisfied with the quality of th...,['The LG product line consistently delivers ex...
563,GC-B22FTLPL,['Performance'],77,61,16,0.21,Customers like the performance of the refriger...,"['Great product, very good price and working f..."


In [113]:
for count,product in enumerate(list(set(df['product_name'].unique()))):
    df = df[df['product_name']==product]
    
    #멘션 많은 키워드
    df_m = df[df['mention']==max(df['mention'])]
    
    #부정리뷰 비율 높은 키워드
    df_n = df[df['neg_percent']==max(df['neg_percent'])]

    
    if count==0:
        df_max = pd.concat([df_m, df_n], axis=0, ignore_index=True)
    else:
        df_new = pd.concat([df_m, df_n], axis=0, ignore_index=True)
        df_max = pd.concat([df_max, df_new], axis=0, ignore_index=True)
    


ValueError: max() arg is an empty sequence

In [79]:
#멘션 많은 키워드
df[df['mention']==max(df['mention'])]

#부정리뷰 비율 높은 키워드
df[df['neg_percent']==max(df['neg_percent'])]

,product_name,keyword,mention,positive,negative,neg_percent,comment,examples
7,GL-I292RPZX,['Scratching'],44,6,38,0.86,Customers are dissatisfied with the scratching...,['...hour I just opened the boxes then I found...


### 4) csv 형태로 저장

In [13]:
# total_review_df.to_csv('Amazon_LG_TV_review.csv')
# total_review_df.to_csv('Amazon_LG_Washing_Machines_review.csv')
# total_review_df.to_csv('Amazon_LG_Air_Conditioner_review.csv')
total_review_df.to_csv('Amazon_LG_Refrigerator_review.csv')